# Object Detection on COCO Dataset

> Example of training an object detection model on COCO dataset

In [ ]:
import torch
import torchvision
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
from PIL import Image

from objdetect.data import ObjectDetectionDataset, get_detection_transforms
from objdetect.models import create_model
from objdetect.learner import ObjectDetectionLearner, SaveModelCallback
from objdetect.core import plot_boxes

## Load COCO Dataset

First, we'll load the COCO dataset. You should download the COCO dataset from [https://cocodataset.org](https://cocodataset.org) and extract it to a local directory.

In [ ]:
# Set path to COCO dataset
coco_path = Path('path/to/coco')

# Create transforms for training and validation
train_transforms = get_detection_transforms(train=True, size=640)
val_transforms = get_detection_transforms(train=False, size=640)

# Load COCO datasets
train_ds = ObjectDetectionDataset.from_coco(coco_path, splits=['train'], transforms=train_transforms)
val_ds = ObjectDetectionDataset.from_coco(coco_path, splits=['val'], transforms=val_transforms)

In [ ]:
# Check dataset
print(f"Number of training samples: {len(train_ds)}")
print(f"Number of validation samples: {len(val_ds)}")
print(f"Number of classes: {train_ds.num_classes}")
if train_ds.class_names:
    print(f"Class names: {train_ds.class_names}")

In [ ]:
# Visualize a sample from the dataset
sample_idx = 0
train_ds.show_sample(sample_idx)

## Create Model and Learner

In [ ]:
# Create Faster R-CNN model
model = create_model(model_name="faster_rcnn", num_classes=train_ds.num_classes,
                     backbone_name="resnet50", pretrained_backbone=True)

# Create learner
learner = ObjectDetectionLearner(
    dataset=(train_ds, val_ds),
    model=model,
    batch_size=2,
    num_workers=2,
    callbacks=[SaveModelCallback(save_path='checkpoints', save_name='faster_rcnn')]
)

## Train the Model

In [ ]:
# Train with one cycle policy
learner.fit_one_cycle(n_epochs=10, max_lr=1e-4)

## Make Predictions

In [ ]:
# Make prediction on a sample image
img_idx = 5  # Choose a sample from validation set
img, _ = val_ds[img_idx]

# Predict
pred = learner.predict(img, threshold=0.5)

# Show results
learner.show_results(img, pred)

In [ ]:
# Predict on an external image
external_img_path = 'path/to/image.jpg'
learner.show_results(external_img_path)

## Save and Load Model

In [ ]:
# Save model manually
torch.save(learner.model.state_dict(), 'faster_rcnn_coco.pth')

# Load model
new_model = create_model(model_name="faster_rcnn", num_classes=train_ds.num_classes)
new_model.load_state_dict(torch.load('faster_rcnn_coco.pth'))

# Create a new learner with loaded model
new_learner = ObjectDetectionLearner(
    dataset=(train_ds, val_ds),
    model=new_model,
    batch_size=2
)

## Evaluate Model

In [ ]:
# Evaluate on validation set
def evaluate(learner, dataset, threshold=0.5, iou_threshold=0.5):
    """Simple evaluation function."""
    learner.model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for idx in range(len(dataset)):
            img, target = dataset[idx]
            pred = learner.predict(img, threshold=threshold)
            
            # TODO: Calculate metrics like mAP
            # This is a placeholder for actual evaluation
            
    return {"placeholder": 0.0}

In [ ]:
# Run evaluation
metrics = evaluate(learner, val_ds)
print(f"Evaluation metrics: {metrics}")